# TODO
- TF/IDF
- get ratings
- partial tags match (female harem matches harem)
- get more data
- iteractive fine tuning of the search (showing candiate tags)
- store and remember (profile, liked animes in the past)
- scrape 'sources' -> ratings
- ai turn (scraped) summary to tags
- age
- additional criteria (episodes, seasons, etc)

In [1]:
import pandas as pd 
import json

In [2]:
with open("./anime-offline-database.json", "r") as file:
    jsondata = json.load(file)

In [17]:
items = jsondata['data']
print(items[1])

{'sources': ['https://anidb.net/anime/10143', 'https://anilist.co/anime/102416', 'https://anime-planet.com/anime/chiaki-kuriyama-0', 'https://anisearch.com/anime/9010', 'https://kitsu.app/anime/8925', 'https://myanimelist.net/anime/20707', 'https://notify.moe/anime/Ff1bpKmmR'], 'title': '"0"', 'type': 'SPECIAL', 'episodes': 1, 'status': 'FINISHED', 'animeSeason': {'season': 'SUMMER', 'year': 2013}, 'picture': 'https://cdn.myanimelist.net/images/anime/12/81160.jpg', 'thumbnail': 'https://cdn.myanimelist.net/images/anime/12/81160t.jpg', 'duration': {'value': 240, 'unit': 'SECONDS'}, 'synonyms': ['"Zero"', '0 (Zero)', 'Chiaki Kuriyama - 0', 'Chiaki Kuriyama - Zero', 'Chiaki Kuriyama: "0"', 'Chiaki Kuriyama: 「0」', 'Kuriyama Chiaki - 0', '「0」', '栗山 千明「0」', '栗山千明 - 0'], 'relatedAnime': [], 'tags': ['drama', 'female protagonist', 'indefinite', 'music', 'present']}


In [4]:
rows = []
for item in items:
    tup = (
    item['title'],
    item['type'],
    item['tags'],
    item['episodes']
        )
    rows.append(tup)

In [5]:
print(len(rows))

34955


In [6]:
df = pd.DataFrame(rows, columns=["title", "type", "tags", "episodes"])

In [7]:
df

,title,type,tags,episodes
0,!NVADE SHOW!,SPECIAL,"[band, full cgi, music, primarily female cast,...",1
1,"""0""",SPECIAL,"[drama, female protagonist, indefinite, music,...",1
2,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",MOVIE,"[family friendly, fantasy, frogs, kids]",1
3,"""Ai"" wo Taberu",MOVIE,[drama],1
4,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",OVA,"[asia, based on a light novel, comedy, drama, ...",1
...,...,...,...,...
34950,▲MEW▲△MEW△CAKE,SPECIAL,[],1
34951,◯,ONA,"[abstract, angst, avant garde, black and white...",1
34952,「S.K.T(Swallow Knights Tales)」 Weptun Teureill...,SPECIAL,"[action, comedy, fantasy]",1
34953,空の境界,TV,"[action, mystery, romance, supernatural, suspe...",13


In [8]:
has_tags = 0
tv_shows = 0
all_tags = []
for idx, row in df.iterrows():
    if row['type'] != "TV":
        continue
    tv_shows += 1
    tags = row['tags']
    if len(tags) > 0:
        has_tags += 1

        for tag in tags:
            if not tag in all_tags:
                all_tags.append(tag)
        
print("tv shows", tv_shows)
print("rows with tags", has_tags)
print("tags", len(all_tags))

tv shows 8880
rows with tags 8818
tags 1147


In [9]:
print(all_tags[:20])

['acting', 'age regression', 'anti-hero', 'award winning', 'based on a manga', 'bowling', 'bullying', 'coming of age', 'daily life', 'dancing', 'denpa', 'detective', 'drama', 'ensemble cast', 'fake relationship', 'family life', 'female harem', 'female protagonist', 'filmmaking', 'genius']


In [10]:
df['episodes'].mean()

np.float64(13.0160205979116)

In [11]:
df['episodes'].max()

np.int64(3057)

# find common tags

In [12]:
tags_to_match = ['adventure', 'fantasy', 'male', 'romance']

In [13]:
def find_matches(search_tags,anime_tags):
    matches = 0
    for tag in search_tags:
        for anime_tag in anime_tags:
            if tag == anime_tag:
                matches += 1
                continue

            if tag in anime_tag:
                matches += 0.5
                continue
                
    match_ratio = matches*100/len(anime_tags)
    return matches, match_ratio

In [14]:
matching_animes = []

for idx, row in df.iterrows():
    if row['type'] != "TV":
        continue
    tags = row['tags']
    if len(tags) == 0:
        continue
    matches, match_ratio = find_matches(tags_to_match, tags)
    if matches == 0:
        continue    
    anime = (row['title'], matches, match_ratio)
    matching_animes.append(anime)

print(len(matching_animes))

6207


In [15]:
matching_df = pd.DataFrame(matching_animes, columns=['title', 'matches', 'ratio'])

In [16]:
matching_df.sort_values(["matches", "ratio"], ascending=[False, False])

,title,matches,ratio
3297,Mato Seihei no Slave,6.5,13.829787
2912,Kyou kara Maou! 3rd Series,6.0,17.647059
1255,Dungeon ni Deai wo Motomeru no wa Machigatteir...,6.0,14.634146
1253,Dungeon ni Deai wo Motomeru no wa Machigatteir...,6.0,12.244898
5041,Sword Art Online II,6.0,10.714286
...,...,...,...
6022,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu ...,0.5,0.961538
2350,Joker Game,0.5,0.925926
608,Boku dake ga Inai Machi,0.5,0.909091
1036,Danganronpa: Kibou no Gakuen to Zetsubou no Ko...,0.5,0.862069
